# Probando por separado

In [1]:
from cmp.evaluation import evaluate_reverse_parse

from engine.lexer.hulk_lexer import HulkLexer 
from engine.language.tokens_type import hulk_tokens
from engine.language.grammar import G
from engine.parser.hulk_parser import HulkParser
import dill
import sys

try:
    with open('engine/lexer/hulk_lexer.pkl', 'rb') as hulk_lexer_dp:
        hulk_lexer = dill.load(hulk_lexer_dp)
except:
    sys.setrecursionlimit(10000)
    hulk_lexer = HulkLexer(hulk_tokens, G.EOF)
    with open('engine/lexer/hulk_lexer.pkl', 'wb') as hulk_lexer_dp:
        dill.dump(hulk_lexer, hulk_lexer_dp)


try:
    with open('engine/parser/hulk_parser.pkl', 'rb') as hulk_parser_dp:
        hulk_parser = dill.load(hulk_parser_dp)
except:
    sys.setrecursionlimit(10000)
    hulk_parser = HulkParser()
    with open('engine/parser/hulk_parser.pkl', 'wb') as hulk_parser_dp:
        dill.dump(hulk_parser, hulk_parser_dp)

print(hulk_parser.tokens_terminals_map)

{'$': '$', <TokenType.EOF: 56>: '$', '$': '$', <TokenType.OPAR: 5>: '(', <TokenType.CPAR: 6>: ')', <TokenType.OPCOR: 9>: '[', <TokenType.CLCOR: 10>: ']', <TokenType.OPCUR: 7>: '{', <TokenType.CLCUR: 8>: '}', <TokenType.COMMA: 1>: ',', <TokenType.DOT: 2>: '.', <TokenType.COLON: 3>: ':', <TokenType.SEMI: 4>: ';', <TokenType.ARROW: 49>: '=>', <TokenType.DOUBLE_OR: 54>: '||', <TokenType.EQUAL: 26>: '=', <TokenType.DESTRUCTIVE: 55>: ':=', <TokenType.IDX: 11>: 'id', <TokenType.STRING: 12>: 'string', <TokenType.NUM: 13>: 'number', <TokenType.BOOLEAN: 14>: 'boolean', <TokenType.PLUS: 15>: '+', <TokenType.MINUS: 16>: '-', <TokenType.STAR: 17>: '*', <TokenType.DIV: 18>: '/', <TokenType.MOD: 19>: '%', <TokenType.POT: 20>: '^', <TokenType.POW2: 48>: '**', <TokenType.AND: 21>: '&', <TokenType.OR: 22>: '|', <TokenType.NOT: 23>: '!', <TokenType.CONCAT: 24>: '@', <TokenType.CONCAT_SP: 25>: '@@', <TokenType.EQUAL_EQUAL: 50>: '==', <TokenType.NOT_EQUAL: 51>: '!=', <TokenType.LESS_EQUAL: 53>: '<=', <Toke

In [2]:
from tests.test_cases import test_cases

tokens = []
right_parses = []
operations = []
asts = []

for i,input in enumerate(test_cases):
    token = hulk_lexer(input)
    tokens.append(token)

    right_parse, operation = hulk_parser(token)
    right_parses.append(right_parse)
    operations.append(operation)

    ast = evaluate_reverse_parse(right_parse, operation, token)
    asts.append(ast)

    print(f"Test {i} loaded successfully.")
    # print(token)
    # print(right_parse)
    # print(operation)
    # print(ast)
    # print("="*100)


Test 0 loaded successfully.
Test 1 loaded successfully.
Test 2 loaded successfully.
Test 3 loaded successfully.
Test 4 loaded successfully.
Test 5 loaded successfully.
Test 6 loaded successfully.
Test 7 loaded successfully.
Test 8 loaded successfully.
Test 9 loaded successfully.
Test 10 loaded successfully.
Test 11 loaded successfully.


In [3]:

from engine.semantic.type_builder_visitor import TypeBuilder
# from engine.semantic.type_checker_visitor import TypeChecker
from engine.semantic.type_collector_visitor import TypeCollector
from engine.semantic.type_inferer_visitor import TypeInferer
from engine.semantic.variable_collector_visitor import VarCollector


def semantic_analysis_pipeline(ast, debug=False):
    # if debug:
    #     formatter = Formatter()
    #     formatted_ast = formatter.visit(ast)
    #     print('===================== AST =====================')
    #     print(formatted_ast)
    if debug:
        print('============== COLLECTING TYPES ===============')
    errors = []
    collector = TypeCollector(errors)
    collector.visit(ast)
    context = collector.context
    if debug:
        print('Errors: [')
        for error in errors:
            print('\t', error)
        print(']')
        print('Context:')
        print(context)
        print('=============== BUILDING TYPES ================')
    builder = TypeBuilder(context, errors)
    builder.visit(ast)
    if debug:
        print('Errors: [')
        for error in errors:
            print('\t', error)
        print(']')
        print('Context:')
        print(context)
        print('=============== CHECKING TYPES ================')
        print('---------------- COLLECTING VARIABLES ------------------')
    var_collector = VarCollector(context, errors)
    scope = var_collector.visit(ast)
    if debug:
        print('Errors: [')
        for error in errors:
            print('\t', error)
        print(']')
        print('Context:')
        print(context)
        print('Scope:')
        print(scope)
        print('---------------- INFERRING TYPES ------------------')
    type_inferrer = TypeInferer(context, errors)
    type_inferrer.visit(ast)

    # Check if there are any inference errors and change the types to ErrorType if there are
    # inference_errors = context.inference_errors() + scope.inference_errors()
    # errors.extend(inference_errors)
    if debug:
        # print('Iterations: ' + str(type_inferrer.current_iteration))
        print('Errors: [')
        for error in errors:
            print('\t', error)
        print(']')
        print('Context:')
        print(context)
        print('Scope:')
        print(scope)
        print('---------------- CHECKING TYPES ------------------')
    # type_checker = TypeChecker(context, errors)
    # type_checker.visit(ast)
    # if debug:
    #     print('Errors: [')
    #     for error in errors:
    #         print('\t', error)
    #     print(']')
        # print('Context:')
        # print(context)
        # print('Scope:')
        # print(scope)
    return ast, errors, context, scope


In [ ]:
print(semantic_analysis_pipeline(asts[0],debug=True))

In [4]:
print(semantic_analysis_pipeline(asts[8],debug=True))

============== COLLECTING TYPES ===============
Errors: [
]
Context:
{
	type Unknow {}
	
	type Object {
		[method] equals(other:Object): Boolean;
		[method] toString(): String;
	}
	
	type Number : Object {}
	
	type Boolean : Object {}
	
	type String : Object {
		[method] size(): Number;
		[method] next(): Boolean;
		[method] current(): String;
	}
	
	type Range : Object {
		[attrib] min : Number;
		[attrib] max : Number;
		[attrib] current : Number;
		[method] next(): Boolean;
		[method] current(): Number;
	}
	
	type PolarPoint {}
	
}
=============== BUILDING TYPES ================
visitando el tipo PolarPoint
el current type es PolarPoint
visitando un atributo y
el tipo de atributo asignado es Unknow
visitando un atributo x
el tipo de atributo asignado es Unknow
visitando un atributo strin
el tipo de atributo asignado es Unknow
3
visitando un metodo retornaphi
el tipo de retorno del metodo es Unknow
Errors: [
]
Context:
{
	type Unknow {}
	
	type Object {
		[method] equals(other:Object)